In [1]:
pip install tweepy

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Use your own credentials here
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAALH2sAEAAAAA%2FNTBYJr9PqJwKKGlZjpQMhCtOyc%3Db70RXl7oqeU8r9sGgWorvjTYps6mr4ZHllBDi6Z0oaq3xGnVRu"


In [4]:
import tweepy

# Authenticate with your API keys
bearer_token = "AAAAAAAAAAAAAAAAAAAAALH2sAEAAAAA%2FNTBYJr9PqJwKKGlZjpQMhCtOyc%3Db70RXl7oqeU8r9sGgWorvjTYps6mr4ZHllBDi6Z0oaq3xGnVRu"

# Define the client
client = tweepy.Client(bearer_token=bearer_token)

# Set up query parameters
query = "mental health OR anxiety OR depression -is:retweet lang:en"

# Fetch tweets
response = client.search_recent_tweets(query=query, tweet_fields=["author_id", "created_at", "text"], max_results=100)

# Print collected tweets
for tweet in response.data:
    print(f"Tweet ID: {tweet.id}")
    print(f"Author ID: {tweet.author_id}")
    print(f"Date: {tweet.created_at}")
    print(f"Text: {tweet.text}")
    print("-" * 50)


Tweet ID: 1892596267026383152
Author ID: 1800872197184999425
Date: 2025-02-20 15:24:47+00:00
Text: @al1most Men are depression it self
--------------------------------------------------
Tweet ID: 1892596266237821169
Author ID: 1781077020778356739
Date: 2025-02-20 15:24:47+00:00
Text: @tedcruz With a child that young, it is the mother that is the problem, she wanted a daughter instead of a son.
I am sure that baby BOY didn't do up HIS own hair to look like a girl.
(Or pick out the clothes to dress as a girl.) The "Mother" needs mental health treatment. 🙄
--------------------------------------------------
Tweet ID: 1892596265822609734
Author ID: 50532792
Date: 2025-02-20 15:24:47+00:00
Text: RT @NiallHarbison: This was me 4 years ago. I was in ICU in hospital as I’d just nearly drunk myself to death. 

I’d been an alcoholic and…
--------------------------------------------------
Tweet ID: 1892596262437822869
Author ID: 926721216441962496
Date: 2025-02-20 15:24:46+00:00
Text: RT @Ore_akii

In [1]:
import tweepy
import re
import pandas as pd
import os

# Twitter API Credentials
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAALH2sAEAAAAA%2FNTBYJr9PqJwKKGlZjpQMhCtOyc%3Db70RXl7oqeU8r9sGgWorvjTYps6mr4ZHllBDi6Z0oaq3xGnVRu"

# Authenticate with Twitter API
client = tweepy.Client(bearer_token=BEARER_TOKEN)

# Define mental health categories and keywords
mental_health_labels = {
    "Anxiety": ["anxiety", "panic attack", "nervous", "overthinking", "restless", "fear", "phobia"],
    "Depression": ["depression", "hopeless", "worthless", "empty", "suicidal", "low mood", "tearful"],
    "Stress": ["stress", "pressure", "overwhelmed", "exhausted", "tired", "burnout", "tense"],
    "PTSD": ["ptsd", "flashbacks", "trauma", "nightmares", "intrusive thoughts", "hypervigilance"],
    "OCD": ["ocd", "obsessive thoughts", "compulsions", "rituals", "intrusive thoughts"],
    "Bipolar Disorder": ["bipolar", "manic", "hypomania", "mood swings", "extreme energy"],
    "Schizophrenia": ["schizophrenia", "hallucinations", "delusions", "paranoia", "disorganized thinking"]
}

# Function to assign label based on keywords
def assign_label(text):
    text = text.lower()
    for label, keywords in mental_health_labels.items():
        if any(word in text for word in keywords):
            return label
    return "Unknown"

# Function to clean tweet text
def clean_tweet(text):
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)  # Remove URLs
    text = re.sub(r"@\w+", "", text)  # Remove mentions
    text = re.sub(r"#\w+", "", text)  # Remove hashtags
    text = re.sub(r"[^A-Za-z0-9\s]", "", text)  # Remove special characters
    return text.lower().strip()

# Load existing tweet IDs to prevent duplicates
csv_file = "mental_health_tweets_labeled.csv"

if os.path.exists(csv_file):
    existing_df = pd.read_csv(csv_file)
    existing_tweet_ids = set(existing_df["tweet_id"].astype(str))
else:
    existing_tweet_ids = set()

# Search for tweets
query = "anxiety OR depression OR stress OR ptsd OR ocd OR schizophrenia -is:retweet lang:en"
tweets = client.search_recent_tweets(query=query, tweet_fields=["id", "created_at", "text"], max_results=100)

# Store tweets in a structured format (avoid duplicates)
tweets_data = []
for tweet in tweets.data:
    tweet_id = str(tweet.id)
    if tweet_id not in existing_tweet_ids:  # Avoid duplicate tweets
        cleaned_text = clean_tweet(tweet.text)
        label = assign_label(cleaned_text)
        tweets_data.append({
            "tweet_id": tweet_id,
            "created_at": tweet.created_at.strftime("%Y-%m-%d"),
            "text": tweet.text,
            "cleaned_text": cleaned_text,
            "label": label
        })

# Save only new data
if tweets_data:
    new_df = pd.DataFrame(tweets_data)
    if os.path.exists(csv_file):
        new_df.to_csv(csv_file, mode='a', header=False, index=False)  # Append new data
    else:
        new_df.to_csv(csv_file, index=False)  # Create file if it doesn't exist
    print(f"✅ {len(tweets_data)} new tweets saved!")
else:
    print("⚠️ No new unique tweets found.")




ConnectionError: HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /2/tweets/search/recent?tweet.fields=id%2Ccreated_at%2Ctext&max_results=100&query=anxiety+OR+depression+OR+stress+OR+ptsd+OR+ocd+OR+schizophrenia+-is%3Aretweet+lang%3Aen (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000028EF7C2B4D0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))